# Training DNN with Plain Tensorflow

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
x = np.random.rand(3)
print(type(x), x.shape)

y = np.random.rand(3, 1)
print(type(y), y.shape)

x, y

<class 'numpy.ndarray'> (3,)
<class 'numpy.ndarray'> (3, 1)


(array([0.81494065, 0.00265547, 0.78473677]), array([[0.86465187],
        [0.4243878 ],
        [0.33213319]]))

## Mnist Dataset

In [3]:
mnist = tf.keras.datasets.mnist 

In [29]:
from tensorflow.examples.tutorials.mnist import input_data

In [61]:
(X_train, y_train), (X_test, y_test)  = mnist.load_data()

In [76]:
print(X_train.shape)
# X_train1 = X_train.reshape((1, 1, -1))
# print(X_train1.shape)
# print(X_train1.reshape())

tmp = X_train.reshape(-1, 784)
tmp.shape
tmp1 = tmp.reshape(-1, 784)

print(tmp1.shape)


tmp1

(60000, 28, 28)
(60000, 784)


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [36]:
tmp = X_train.reshape(-1, 1, -1)
tmp.shape

ValueError: can only specify one unknown dimension

In [21]:
X_train[0].shape

(28, 28)

In [7]:
print(y_train.shape)
print(y_train[2])

(60000,)
4


In [8]:
np.random.rand(3, 4)

array([[0.74439687, 0.53459384, 0.77273977, 0.25084442],
       [0.831738  , 0.88741674, 0.64189195, 0.92634429],
       [0.34762873, 0.06705145, 0.77330967, 0.85008435]])

# Plain Tensorflow Implementation

### Initial Declaration

In [9]:
n_inputs = 28 * 28 
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

# Construction Phase
- The Place Holder X acts as a Input layer, it will replaced with 1 training batch at a time 
- 2 hidden identical hidden layers , vaires only in INPUT and number of Neurons
- Output layer , similar to hidden layer but uses softmax activation function

### Placeholders to change Features and respective target while training
    - Since this placeholder will have different instances , i.e different batch size for each iteration, it is declared None now

In [10]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [11]:
reset_graph()
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name ="y")

In [12]:
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="kernel")
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z
            

# Neuron Structure

In [13]:
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1",
                           activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2",
                           activation=tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name="outputs")

In [14]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                              logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [15]:

learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [16]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [17]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [18]:
n_epochs = 40
batch_size = 50

In [19]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [77]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            print(X_batch.shape, y_batch.shape)
            X_batch = X_batch.reshape(-1,784)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Batch accuracy:", acc_batch, "Val accuracy:", acc_val)

    save_path = saver.save(sess, "./my_model_final.ckpt")

(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28)

(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28)

(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28) (50,)
(50, 28, 28)

NameError: name 'X_valid' is not defined

In [ ]:
# with tf.name_scope("dnn"): 
#     hidden1 = neuron_layer(X, n_hidden1, name ="hidden1", activation = tf.nn.relu) 
#     hidden2 = neuron_layer(hidden1, n_hidden2, name ="hidden2", activation = tf.nn.relu) 
#     logits = neuron_layer(hidden2, n_outputs, name ="outputs")

# with tf.name_scope("loss"): 
#     xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = logits) 
#     loss = tf.reduce_mean( xentropy, name ="loss")

# learning_rate = 0.01 
# with tf.name_scope("train"): 
#     optimizer = tf.train.GradientDescentOptimizer(learning_rate) 
#     training_op = optimizer.minimize(loss)

# with tf.name_scope("eval"): 
#     correct = tf.nn.in_top_k(logits, y, 1) 
#     accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
print("Nandha")

In [ ]:
init = tf.global_variables_initializer() 
saver = tf.train.Saver()



In [ ]:
from sklearn.neural_network.